In [1]:
from darts import TimeSeries
import numpy as np
from darts.models import RNNModel
from darts.utils.likelihood_models import LaplaceLikelihood
import darts.utils.timeseries_generation as tg
import os

os.environ["CUDA_VISIBLE_DEVICES"]="1"
# Creating time series to train on
length = 400
trend = tg.linear_timeseries(length=length, end_value=4)
season1 = tg.sine_timeseries(length=length, value_frequency=0.05, value_amplitude=1.0)
noise = tg.gaussian_timeseries(length=length, std=0.6)
noise_modulator = (
    tg.sine_timeseries(length=length, value_frequency=0.02)
    + tg.constant_timeseries(length=length, value=1)
) / 2
noise = noise * noise_modulator

target_series = sum([noise, season1])
covariates = noise_modulator
target_train, target_val = target_series.split_after(0.65)

# Initiating and fitting a RNN
rnn_oxy = RNNModel(model="LSTM",
                    hidden_dim=64,
                    dropout=0,
                    batch_size=32,
                    input_chunk_length=30,
                    output_chunk_length=34,
                    likelihood=LaplaceLikelihood(),
                    n_rnn_layers=2,
                    random_state=0)

rnn_oxy.fit(target_train, 
             epochs=1, 
             verbose=True)

/home/rstudio/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
ignoring user defined `output_chunk_length`. RNNModel uses a fixed `output_chunk_length=1`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name          | Type             | Params
---------------------------------------------------
0 | criterion     | MSELoss          | 0     
1 | train_metrics | MetricCollection | 0     
2 | val_metrics   | MetricCollection | 0     
3 | rnn           | LSTM             | 50.4 K
4 | V             | Linear           | 130   
---------------------------------------------------
50.6 K    Trainable params
0         Non-trainable params
50.6 K    Total para

Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 45.26it/s, train_loss=1.280]

`Trainer.fit` stopped: `max_epochs=1` reached.


Epoch 0: 100%|██████████| 8/8 [00:00<00:00, 44.88it/s, train_loss=1.280]


RNNModel(model=LSTM, hidden_dim=64, n_rnn_layers=2, dropout=0, training_length=24, batch_size=32, input_chunk_length=30, output_chunk_length=34, likelihood=<darts.utils.likelihood_models.LaplaceLikelihood object at 0x7ff254f33b50>, random_state=0)